In [1]:
!pip3 install virtualenv
!virtualenv ai_course_env

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 10.9 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 2749ms
  creator CPython3Posix(dest=/content/ai_course_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==74.1.2, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
# mount Google Drive to Colab session
from google.colab import drive
drive.mount('/content/drive')

# copy to session drive ('/content')
!cp -r /content/drive/MyDrive/GoogleNews-vectors-negative300.bin /content

Mounted at /content/drive


In [3]:
# load word2vec model
import os
gn_vec_path = os.path.abspath(os.getcwd()) + "/GoogleNews-vectors-negative300.bin"

from gensim.models import Word2Vec, KeyedVectors
model = KeyedVectors.load_word2vec_format(gn_vec_path, binary=True) #load the model

# Using pre-trained word embeddings

In [14]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")
model.most_similar([model['king']], topn=11)

[=============================================-----] 90.4% 59.7/66.0MB downloaded

KeyboardInterrupt: 

# Training a song embedding model

In [15]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

In [16]:
# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata

lines = data.read().decode("utf-8").split('\n')[2:]

In [17]:
lines

['0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 2 42 43 44 45 46 47 48 20 49 8 50 51 52 53 54 55 56 57 25 58 59 60 61 62 3 63 64 65 66 46 47 67 2 48 68 69 70 57 50 71 72 53 73 25 74 59 20 46 75 76 77 59 20 43 ',
 '78 79 80 3 62 81 14 82 48 83 84 17 85 86 87 88 74 89 90 91 4 73 62 92 17 53 59 93 94 51 50 27 95 48 96 97 98 99 100 57 101 102 25 103 3 104 105 106 107 47 108 109 110 111 112 113 25 63 62 114 115 84 116 117 118 119 120 121 122 123 50 70 71 124 17 85 14 82 48 125 47 46 72 53 25 73 4 126 59 74 20 43 127 128 129 13 82 48 130 131 132 133 134 135 136 137 59 46 138 43 20 139 140 73 57 70 141 3 1 74 142 143 144 145 48 13 25 146 50 147 126 59 20 148 149 150 151 152 56 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 60 176 51 177 178 179 180 181 182 183 184 185 57 186 187 188 189 190 191 46 192 193 194 195 196 197 198 25 199 200 49 201 100 202 203 204 205 206 207 32 208 20

In [18]:
# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

In [19]:
playlists[:1]

[['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '2',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '20',
  '49',
  '8',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '25',
  '58',
  '59',
  '60',
  '61',
  '62',
  '3',
  '63',
  '64',
  '65',
  '66',
  '46',
  '47',
  '67',
  '2',
  '48',
  '68',
  '69',
  '70',
  '57',
  '50',
  '71',
  '72',
  '53',
  '73',
  '25',
  '74',
  '59',
  '20',
  '46',
  '75',
  '76',
  '77',
  '59',
  '20',
  '43']]

In [20]:
# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [21]:
songs_df.head()

,title,artist
id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow


In [22]:
print(f"Playlist #0: {playlists[0]}")
print(f"Playlist #1: {playlists[1]}")

Playlist #0: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43']
Playlist #1: ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118'

In [ ]:
# train model
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(sentences=playlists, window=20, negative=50, min_count=1, workers=4)

In [ ]:
song_id = 2172
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [ ]:
def print_recommendations(song_id):
    similar_songs = model.wv.most_similar(positive=str(song_id),topn=5)
    similar_song_ids = [_id for _id, _ in similar_songs]
    return songs_df.iloc[similar_song_ids]

In [ ]:
print_recommendations(song_id)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
5586,The Last In Line,Dio
5549,November Rain,Guns N' Roses
2976,I Don't Know,Ozzy Osbourne
3167,Unchained,Van Halen
